In [13]:
from typing import Any, Type
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from datetime import datetime
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper

llm = ChatOpenAI(temperature=0.1)

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for"
    )

class SaveToFileToolArgsSchema(BaseModel):
    content: str = Field(
        description="The content to save to the file"
    )
    filename: str = Field(
        default="research_results",
        description="The filename where the content will be saved"
    )

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to search on Wikipedia.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaAPIWrapper()
        return wikipedia.run(query)

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to search on DuckDuckGo.

    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class SaveToFileTool(BaseTool):
    name = "SaveToFileTool"
    description = """
    Use this tool to save the text to a file.
    """

    args_schema: Type[
        SaveToFileToolArgsSchema
    ] = SaveToFileToolArgsSchema

    def _run(self, content: str, filename):
        filename_formatted = f"{filename}_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"
        file_path = f"./researchfiles/{filename_formatted}"
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
        return f"File saved {filename_formatted}"


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        SaveToFileTool(),
    ],
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Run the agent with this query: "{query}", 
            The agent attempts to search DuckDuckGo and Wikipedia, extracts the content, 
            and then Save your research as a .txt file.
            """,
        )
    ]
)

chain = {"query": RunnablePassthrough()} | prompt | agent

In [14]:
query = "Research about the XZ backdoor"
results = chain.invoke({"query":query})



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'Research about the XZ backdoor'}`


Getty Images. 207. On Friday, a lone Microsoft developer rocked the world when he revealed a backdoor had been intentionally planted in xz Utils, an open source data compression utility available ... That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. Threat actors added malicious code (or a backdoor) to the XZ utility versions 5.6.0 and 5.6.1. The installed backdoor works by manipulating the sshd, a server process that facilitates secure internet connections using the SSH protocol. The sshd is responsible for user authentication, encryption, terminal connections, file transfers, and tunneling. On March 28, 2024, a backdoor was identified in the popular xz-utils package and reported to the oss-security@openwall mailing list o